In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [5]:
# Create a DataFrame with user input (A) and response (B)
def create_question_answer_dataframe():
    conversation_data = [
        ["I am so extremely annoyed by my neighbors. They keep playing loud music all the time, and I can’t even hear myself think. I’ve already mentioned it a couple of times, but they just keep playing loud music!", "It sounds like you’re feeling frustrated or even powerless because you have a need for peace and a calm environment to feel comfortable in your own home. Is that correct?"],
        ["Lately, I haven’t been feeling great. I think it’s because I see everyone earning more and advancing in their careers, and I feel like a total failure.", "Do you maybe feel insecurity because you have a need for growth and a clear direction for your future?"],
        ["I’m having arguments with my partner about money. I feel like they’re not contributing enough to our shared household, but it seems like they don’t even care.", "Do you feel frustration or anger about the situation because you’re missing honesty and collaboration in your financial agreements?"],
        ["My manager refuses to involve me in new initiatives. I feel like I’m not being taken seriously at all.", "Are you maybe experiencing frustration or sadness because you have a need for acknowledgment and involvement?"],
        ["I’m in a situation where I no longer feel safe. In the classroom, I feel judged from all sides and sometimes even ignored.", "Do you feel tension or sadness because you have a need for safety and acceptance in the classroom?"],
        ["I was recently helped on the street when I got into an argument with a neighbor. I really appreciated that person stepping in to help me of their own accord.", "It sounds like you feel grateful right now because your need for support and safety was met in that moment. Is that correct?"],
        ["I’m not sure how to express this, but I can’t find my way out of a situation. Two friends are fighting, and I don’t really know what it’s about. Meanwhile, the friend group seems to be falling apart.", "It sounds like a challenging situation for you, with two friends having a conflict that’s affecting the whole group. Would you like to share more about how you feel in this situation so I can try to understand you better?"],
        ["For me, it feels like my hands are tied. I’ve suggested mediation a couple of times, but one of them isn’t open to it. Now we’re all in awkward situations, like one not inviting the other. Things like that are really uncomfortable for me.", "Do you perhaps feel powerless or tense in this situation because you have a need for harmony and connection within your group of friends?"],
        ["Yeah, I guess so. I’d especially like to see some growth toward reconciliation. That doesn’t seem to be happening now.", "Ah, so do you maybe feel more frustration and sadness because you’d really like to see growth in the restoration of connection between your friends?"],
        ["Yeah, especially the sadness is there. I just don’t know what to do anymore—whether I should act or just let it go.", "It sounds like you feel sadness because you have a need for some perspective and clarity about how you could contribute to this situation, maybe even without exhausting yourself. Is that right?"],
        ["Yeah, I think that’s it.", "Okay. Would you like to explore together whether you want to make a request—either of yourself or someone else—to take a step toward that clarity and possible reconciliation?"],
        ["Yes, I’d like that.", "Great. Let’s think of a simple and concrete way to start that conversation."]
    ]

    # Convert to DataFrame
    df = pd.DataFrame(conversation_data, columns=["User Input (A)", "Response (B)"])
    print(df)

    # Save to CSV
    df.to_csv("conversations.csv", index=False)
    print("DataFrame saved to conversations.csv")
    return df

In [17]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import os

# Load environment variables and model
HF_TOKEN = os.getenv('HF_TOKEN')
MODEL_NAME = "ruslanmv/fine_tuned_llama_empathy"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32, token=HF_TOKEN).to(device)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

SYSTEM_PROMPT = """
You are a compassionate assistant trained to help users explore their emotions and unmet needs.
When users share evaluative words or experiences, respond empathetically by identifying associated feelings and unmet needs. Use clear, concise, and natural responses.
Please answer like a professional doctor only.
Example of Answer:
1. User: "I feel betrayed by my friend."
   Response: "It sounds like you're feeling betrayed. This might involve emotions like hurt or disappointment. Do you think this could be related to a need for trust or honesty in your friendship?"

2. User: "No one respects my ideas at work."
   Response: "I hear that you're feeling unheard or unimportant. It seems like this might be connected to a need for respect or acknowledgment. Can you tell me more about how this situation has been affecting you?"

3. User: "I feel invisible in my family."
   Response: "It seems like you're feeling invisible, which could bring up emotions such as sadness or loneliness. This might point to needs like being seen and heard, inclusion, or belonging. Would you like to share more about how this affects you?"
"""

# Function to clean repeated phrases
def clean_repeated_phrases(response):
    sentences = response.split('. ')
    seen = set()
    cleaned_sentences = []
    for sentence in sentences:
        if sentence not in seen:
            seen.add(sentence)
            cleaned_sentences.append(sentence)
    return '. '.join(cleaned_sentences)

# Chat model function
def chat_with_model(prompt, history):
    try:
        conversation = "\n".join([f"User: {u}\nAI: {a}" for u, a in history])
        full_context = f"{SYSTEM_PROMPT}\n\n{conversation}\nUser: {prompt}\nAI:"
        inputs = tokenizer(full_context, return_tensors="pt", padding=True, truncation=True).to(device)
        output = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask, max_new_tokens=200, temperature=0.7, top_p=0.9, repetition_penalty=1.2, pad_token_id=tokenizer.pad_token_id)
        response = tokenizer.decode(output[0], skip_special_tokens=True).strip()
        response_cleaned = clean_repeated_phrases(response.split("\nAI:")[-1].strip())
        history.append((prompt, response_cleaned))
        return history
    except Exception as e:
        error_message = f"Error: {str(e)}"
        history.append((prompt, error_message))
        return history

# Read from conversations.csv
def create_question_answer_dataframe():
    if os.path.exists("conversations.csv"):
        return pd.read_csv("conversations.csv")
    else:
        raise FileNotFoundError("The file 'conversations.csv' was not found.")

# Load the questions from CSV
questions_df = create_question_answer_dataframe()
question_list = questions_df["User Input (A)"].dropna().tolist()  # Ensure no NaNs

# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Chat with Empathetic AI")
    chat_window = gr.Chatbot(label="Chat History")

    with gr.Row():
        user_input = gr.Textbox(label="Your Prompt", placeholder="Type your message here...")
        submit_button = gr.Button("Submit")

    def update_chat_window(prompt, history):
        updated_history = chat_with_model(prompt, history)
        return updated_history, ""  # Reset user input after submit

    submit_button.click(fn=update_chat_window, inputs=[user_input, chat_window], outputs=[chat_window, user_input])
    question_dropdown = gr.Dropdown(choices=question_list, label="Select a Suggested Question")

    def populate_input_with_question(selected_question):
        return selected_question  # Fill user input with selected question

    question_dropdown.change(fn=populate_input_with_question, inputs=question_dropdown, outputs=user_input)

demo.launch(debug=True, share=True)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://70cf0d8e6b585d703b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://70cf0d8e6b585d703b.gradio.live
